- Run in polynomial time
- Somewhat "intelligent"
- 

In [122]:
import random

#We will be going over this code in lecture.


class Graph:

    def __init__(self):
        self.adj = {}

    def are_connected(self, node1, node2):
        for node in self.adj[node1]:
            if node == node2:
                return True
        return False

    def adjacent_nodes(self, node):
        return self.adj[node]

    def add_node(self, node):
        self.adj[node] = []

    def add_edge(self, node1, node2):
        if node1 not in self.adj[node2]:
            self.adj[node1].append(node2)
            self.adj[node2].append(node1)

    def remove_edge(self, node1, node2):
        if node2 in self.adj[node1]:
            self.adj[node1].remove(node2)
        if node1 in self.adj[node2]:
            self.adj[node2].remove(node1)

    def remove_incident_edges(self, node): #remove edges that connect to a node
        for neighbour in self.adj[node]:
            self.adj[neighbour].remove(node)
        self.adj[node] = []

    def has_no_edges(self): #completely no edges in graph
        for node in self.adj:
            if len(self.adj[node]) > 0:
                return False
        return True

    def copy(self): #duplicate graph
        new_G = Graph()
        new_G.adj = self.adj.copy()
        for node in self.adj:
            new_G.adj[node] = self.adj[node].copy()
        return new_G

    def number_of_nodes(self): 
        return len(self.adj)

    def show(self): #generate output
        for i in range(len(self.adj)):
            print("node:", i)
            print(self.adj[i])

        
def vertex_cover(G):
    nodes = list(G.adj.keys())
    best_cover = nodes
    covers = power_set(nodes)
    for cover in covers:
        if is_vertex_cover(G, cover):
            if len(cover) < len(best_cover):
                best_cover = cover
    return best_cover

def is_vertex_cover(G, cover):
    temp_G = G.copy()   
    for node in cover:
        temp_G.remove_incident_edges(node)
    if temp_G.has_no_edges():
        return True
    return False

def power_set(elements):
    if elements == []:
        return [[]]
    return add_to_each(power_set(elements[1:]), elements[0]) + power_set(elements[1:])

def add_to_each(sets, element):
    my_sets = sets.copy()
    for my_set in my_sets:
        my_set.append(element)
    return my_sets

def create_random_graph(n, m):
    G = Graph()
    for i in range(n):
        G.add_node(i)
    edges = create_edge_list(n)
    for _ in range(m):
        edge = random.choice(edges)
        edges.remove(edge)
        G.add_edge(edge[0],edge[1])
    return G

def create_edge_list(n):
    edges = []
    for i in range(n):
        for j in range(i+1,n):
            edges.append((i,j))
    return edges


In [123]:
# Testing given:
print("graph:")
g1 = create_random_graph(5,5)
g1.show()

# print(g1.are_connected(0,1))
# print(g1.adjacent_nodes(3))

# print("break")
# g1.add_node(5)
# print(g1.add_edge(3))
# g1.show()

# g1.remove_incident_edges(0)
# g1.remove_incident_edges(1)
# g1.remove_incident_edges(2)
# g1.remove_incident_edges(3)
# g1.show()
# print(g1.has_no_edges())



graph:
node: 0
[3, 2]
node: 1
[4, 3]
node: 2
[4, 0]
node: 3
[0, 1]
node: 4
[2, 1]


In [124]:
# print("cover: ")
# c1 = vertex_cover(g1)
# print(c1)

# also = [3,1,2]

# print(is_vertex_cover(g1,also))

# edgeList = create_edge_list(5)
# print(edgeList)
    

In [125]:
def edgeListForGraph(g):
    edges = []
    for i in range(g.number_of_nodes()):
        for j in g.adjacent_nodes(i):
            edges.append((i,j))

    for edge in edges:
        a = edge[0]
        b = edge[1]
        # print(a,b)
        for check in edges:
            if (a == check[1] and b == check[0]):
                edges.remove((check[0],check[1]))
    return edges

def nodeListForGraph(g):
    adj = list()
    for i in range(g.number_of_nodes()):
        adj.append((i,g.adjacent_nodes(i)))

    adj.sort(key=lambda x: len(x[1]), reverse=True)

    return adj

# g1.show()
# print(nodeListForGraph(g1))
# sortNodeList(nodeListForGraph(g1))


The principle:
- Empty set of vertex covers
- List of all edges
- Randomly(!!) select an edge in the graph, and add the two nodes on either side to the vertex cover set
- Remove all the edges that propagate from these vertices
- Do this until no edges left

https://en.wikipedia.org/wiki/Vertex_cover#:~:text=In%20graph%20theory%2C%20a%20vertex,every%20edge%20of%20the%20graph.

In [126]:

def vc_approx1(g):

    Cov = set()
    Edges = edgeListForGraph(g)
    # print(Edges)

    while (len(Edges) != 0):

        ranEdge = random.choice(Edges) #can also replace with node 0

        Cov.add(ranEdge[0])
        Cov.add(ranEdge[1])


        if ( len(g.adjacent_nodes(ranEdge[0])) >= len(g.adjacent_nodes(ranEdge[1]))):
            for i in g.adjacent_nodes(ranEdge[0]):
                try:
                    Edges.remove((ranEdge[0],i))
                    # print("removing edge", ranEdge[0], i)
                except:
                    pass
                try:
                    Edges.remove((i,ranEdge[0]))
                    # print("removing edge", i, ranEdge[0])
                except:
                    pass
        else:
            for j in g.adjacent_nodes(ranEdge[1]):
                
                try:
                    Edges.remove((ranEdge[1],j))
                    # print("removing edge", ranEdge[1], j)
                except:
                    pass
                try:
                    Edges.remove((j,ranEdge[1]))        
                    # print("removing edge", j, ranEdge[1])
                except:
                    pass

    vCover = list(Cov)
    return vCover

#uncomment the prints to see whats happening
# print(vc_approx1(g1))

The principle:

- removes leaf nodes

In [127]:
def vc_approx2(g): #O(V*E+2E+V)
    nodes = nodeListForGraph(g)
    return [node for (node,adj) in nodes if len(adj) > 1]
    
# cov = vc_approx2(g1)
# print(cov)
# print(is_vertex_cover(g1,cov))

In [128]:
def vc_approx3(g): #O(V^2+V*E+VlogV+2E+V) i think
    nodes = nodeListForGraph(g)
    cover = [nodes[0][0]]
    i = 1
    while (not is_vertex_cover(g,cover)):
        cover.append(nodes[i][0])
        i += 1
    return cover

# cov = vc_approx3(g1)
# print(cov)
# print(is_vertex_cover(g1,cov))

In [129]:
def minimal_cover(g):

    graph = g.copy()

    Cov = set()

    NodeList = nodeListForGraph(g)
    EdgeList = edgeListForGraph(g)

    print(NodeList)
    print(EdgeList)

    while (len(EdgeList) != 0):

        for k in sorted(NodeList, key=len, reverse=True):

            Cov.add(k)


            pass

            # print (NodeList[k])

        print(1)

        EdgeList.pop()

    # NodeList.pop(2)

    # print(NodeList)

    # while():

    # print(len(graph.adjacent_nodes(1)))

    return

# minimal_cover(g1)

Approximation:

- C* is optimal, C is what my algorithm returns.
- My approach:
    Find the minimal vertex cover by running out vc's 1000-ish times to find the minimal list. This does not guarantee it will be minimal but it should be sufficient for graphs with less than a 1000 nodes

In [130]:
#nodes: 1/4 of max possible amount

def graphConstructor(n):
    #n - nodes

    # m = int( n*((n-1)/2) * 0.25)
    m = n * 2
    if (m > int( n*((n-1)/2))):
        m = int( n*((n-1)/2))

    print ("vertices:",n)
    print ("edges:", m)

    graph = create_random_graph(n,m)

    return graph

def find_minimal_cover(g):

    minimal = list(range(500000)) #max size of list

    for i in range(500):
        current = list()
        current = vc_approx1(g)

        if (len(current) < len(minimal)):
            minimal = current

    return minimal

def run10(f,g):

    total = 0
    avg = 0

    for _ in range(10):
        current = f(g)
        total += len(current)

    avg = (total/10)

    return avg

In [131]:
gProblem = Graph()
gProblem.add_node(0)
gProblem.add_node(1)
gProblem.add_node(2)
gProblem.add_node(3)
gProblem.add_node(4)
gProblem.add_node(5)
gProblem.add_node(6)
gProblem.add_node(7)
gProblem.add_node(8)
gProblem.add_node(9)

gProblem.add_edge(0,7)
gProblem.add_edge(1,2)
gProblem.add_edge(1,4)
gProblem.add_edge(1,5)
gProblem.add_edge(2,9)
gProblem.add_edge(2,4)
gProblem.add_edge(3,4)
gProblem.add_edge(4,8)
gProblem.add_edge(5,8)
gProblem.add_edge(5,9)
gProblem.add_edge(7,8)

# gProblem.show()

# node: 0
# [7]
# node: 1
# [2, 4, 5]
# node: 2
# [1, 9, 4]
# node: 3
# [4]
# node: 4
# [3, 8, 1, 2]
# node: 5
# [8, 9, 1]
# node: 6
# []
# node: 7
# [0, 8]
# node: 8
# [5, 7, 4]
# node: 9
# [5, 2]

vert = find_minimal_cover(gProblem)
print(vert)

print(vertex_cover(gProblem))
print(vc_approx2(gProblem))
print()
# print([7,4,5,2]) - minimal

#[7,8,4,1,9] is optimal but based on how my algo works, it will never return this.
#node 9 and node 1 have some special properties. Can be visible if you draw the graph

[0, 2, 4, 5, 7, 9]
[7, 5, 4, 2]
[4, 1, 2, 5, 8, 7, 9]



In [132]:
#Testing: 

# Try to find minimal, and compare that to the avarage of 5 runs of the approximation. See how those compare:

# Sample run, we need a better algo to actually, correctly find the minimal. No matter the time complexity.

# print(run10(vc_approx1,gProblem))
# print(run10(vc_approx2,gProblem))
# print(run10(vc_approx3,gProblem))
# print(len(vertex_cover(gProblem)))

In [133]:
def logtocsv(samplesSize,run10,minimal,filename):
    with open(filename, 'a') as f: #'w' for write, 'a' for append
        f.write(str(samplesSize))
        f.write(",")
        f.write(str(run10))
        f.write(",")
        f.write(str(minimal))
        f.write('\n')

In [134]:
# gBig = graphConstructor(300)
# # gBig.show()

# print(run10(gBig))
# print(len(find_minimal_cover(gBig)))


def timing():
    i = 1
    while (i < 21): #701 is as far as i went

        graph = graphConstructor(i)

        logtocsv(i,run10(vc_approx3,graph),len(vertex_cover(graph)),"VC3approx.csv")

        if (i < 102):
            i += 1
        else:
            i += 30

timing()

vertices: 1
edges: 0
vertices: 2
edges: 1
vertices: 3
edges: 3
vertices: 4
edges: 6
vertices: 5
edges: 10
vertices: 6
edges: 12
vertices: 7
edges: 14
vertices: 8
edges: 16
vertices: 9
edges: 18
vertices: 10
edges: 20
vertices: 11
edges: 22
vertices: 12
edges: 24
vertices: 13
edges: 26
vertices: 14
edges: 28
vertices: 15
edges: 30
vertices: 16
edges: 32
vertices: 17
edges: 34
vertices: 18
edges: 36
vertices: 19
edges: 38
vertices: 20
edges: 40


In [135]:
# gprob = create_random_graph(20,20)
# # gprob.show()
# print(vertex_cover(gprob))